# Crude (CLUT) Calculation

The purpose of this notebook is to 
1. Do a crude 1-year SPICE calculation of all small bodies (crude look-up table, CLUT).
    * In this example, I will make a crude calculation for the year 2025. 
    * The calculation starts on 2025-02-01 for 350 days (for every 1-day)
    * Since this is a "crude" calculation, any aberration corrections will be ignored (which doubles the calculation speed), and the observer is a geocenter (not a spacecraft - you can change this, but the computation time may be roughly doubled if ``spkcvo`` is used).
    * ~ 6kB/object, so 1.3M objects correspond to ~10 GB.


For doing so, we need a snapshot of the SBDB list & BSP files.

In this example, I used those queried on **UT2023-12-28**. As of writing (2024 Aug), necessary files are availble via my personal Dropbox:
* SBDB query parquet files (~300MB): [link](https://www.dropbox.com/scl/fo/opi6k5b49bky6bomb6gmt/ACBDWV3cEECB2JH0X2GqAog?rlkey=injz3wl48ff7ci68djelbjkd6&dl=0) 
* Horizons-queried BSP files (~46GB): [link](https://www.dropbox.com/scl/fi/9xr7hpxy7b8p1z856623a/spkbsp.zip?rlkey=ffnky4jq3qhw34tqqbylng4ep&dl=0) (user needs to unzip this)

Save them to a certain location, and modify the PATHS variable below accordingly.

In [1]:
import spicetools as spt
import spiceypy as sp
from astropy.time import Time
import ctypes

from pathlib import Path
import pandas as pd
import numpy as np
from pyarrow import dataset as ds

from astropy import units as u

def iterator(it):
    try:
        from tqdm import tqdm
        return tqdm(it)
    except ImportError:
        return it
# ---------------------------------------------------------------------------------------------------------- #
# CHANGE HERE
PATHS = dict(
    # SPICE kernels
    SPICE=dict(
        TLS="$KERNELS/lsk/naif0012.tls",
        GM="$KERNELS/pck/gm_de440.tpc",
        PCK="$KERNELS/pck/pck00011.tpc",
        DE="$KERNELS/de432s.bsp"
    ),
    ORBROOT="../../../../workspace/__Database/orbits",
    SBDB=dict(
        ast="../../../../workspace/__Database/orbits/sbdb_query_20231228/sbdb_a.parq",
        com="../../../../workspace/__Database/orbits/sbdb_query_20231228/sbdb_c.parq",
    ),
    clutparent="clut"
)
# $KERNELS is a magic word for the path to the SPICE kernels included in this repo.
# You can find them in `src/spicetools/kernels` directory of this repo.
# ORBROOT is the parent directory where the BSP files are located. You may freely change it.
# ---------------------------------------------------------------------------------------------------------- #

chunk = 100000  # Number of objects to be saved into a single parquet file
Path(PATHS["clutparent"]).mkdir(parents=True, exist_ok=True)

########## SPICE INITIALIZATION ##########
# Make the SPICE "meta kernel" for this example
TMP_META_PATH = Path("test.mk")
spt.make_meta(*PATHS["SPICE"].values(), output=TMP_META_PATH)
# Load ("furnish" in SPICE)
handle_meta = sp.furnsh(str(TMP_META_PATH))

########## SBDB/BSP DATA (LIKELY BE DOWNLOADED/MANAGED SEPARATELY) ##########
dtypes = {
    "spkid": int, "pdes":str, "full_name": str, "kind": str, "condition_code": str
}
_dataset = ds.dataset(PATHS["SBDB"]["ast"], format="parquet")
# Use filter to take objects with (1) proper U-parameter & (2) proper H magnitude (for flux modeling)
dfa = _dataset.to_table(
    columns=list(dtypes.keys()) + ["H", "G"],
    filter=(ds.field("condition_code").isin(list("0123456789"))) & (ds.field("H") < 100)
)

_dataset = ds.dataset(PATHS["SBDB"]["com"], format="parquet")
# Use filter to take objects meaningful entry...
dfc = _dataset.to_table(
    columns=list(dtypes.keys()) + ["M1", "M2", "K1", "K2"],
    filter=((ds.field("condition_code").isin(list("0123456789"))) & (ds.field("data_arc") > 0)
            & ((ds.field("M1") < 100) | (ds.field("M2") < 100) | (ds.field("K1") < 100) | (ds.field("K2") < 100)))
)

dfs = dict(c=dfc.to_pandas(), a=dfa.to_pandas())
del dfa, dfc

print(f"Number of Comets   : {len(dfs['c'])}\nNumber of Asteroids: {len(dfs['a'])}")

Number of Comets   : 986
Number of Asteroids: 1339872


## Extreme Optimization for SPICE

Since the SPICE used in `spiceypy` is written in C (CSPICE), every `spiceypy` function will have to convert the input Python variable into C-type objects. This actually results in non-negligible overhead for our case because we have a nested for-loop to call the SPICE function for 1.3M objects * 350 timestamps. 

Changing many input parameters to C-types prior to the for-loop actually nearly doubles the computation speed in our case (on my laptop...).

In [2]:
# Precalculate all ET values
TIMES, ETS, ETS_C = spt.times2et(Time("2025-02-01") + np.arange(350)*u.day, return_c=True)

NO_SPK_FILE = []
ERROR_FILE = []

spkgps = spt.spkgps(ref="ECLIPJ2000", obs=399, dummy_lt=True)


def calc_spice(spkids, parent, outpath):
    spkids_used = []
    xs, ys, zs = [], [], []
    for spkid in iterator(spkids):
        fpath = f"{parent}/spk{spkid}.bsp"
        try:
            handle = sp.spklef(fpath)
            targ_pos = []
            _target = ctypes.c_int(int(spkid))
            for _et in ETS_C:
                # Below is an optimized version of sp.stypes.c_vector_to_python
                targ_pos.append(spkgps(_target, _et))

            sp.spkuef(handle)
        except:  # Some unexpected errors
            if not Path(fpath).exists():
                NO_SPK_FILE.append(spkid)
                continue
            ERROR_FILE.append(spkid)
            continue

        spkids_used.append(spkid)
        targ_pos = np.array(targ_pos).astype(np.float32)
        x, y, z = targ_pos.T
        xs.append(x)
        ys.append(y)
        zs.append(z)
        del x, y, z, targ_pos

    xyz = np.hstack([xs, ys, zs]).astype(np.float32)
    del xs, ys, zs
    df = pd.DataFrame(xyz, columns=[str(i) for i in range(xyz.shape[1])])
    df.insert(loc=0, value=np.array(spkids_used).astype(np.int32), column="spkid")
    df.to_parquet(outpath)
    del df
    return


In [3]:
for ac in "ca":
    _df = dfs[ac]
    bspparent = f"{PATHS['ORBROOT']}/spkbsp/{ac}"
    for i in range(len(_df)//chunk + 1):
        _t = Time.now()
        outpath = f"{PATHS['clutparent']}/{ac:s}_chunk_{i:03d}.parq"
        _spkids = _df["spkid"][i*chunk:(i+1)*chunk]
        calc_spice(_spkids, bspparent, outpath)
        print(f"Saved to {outpath}: N_obj={len(_spkids)}; took {(Time.now() - _t).value*86400:.2f} sec")

100%|██████████| 986/986 [00:02<00:00, 445.33it/s]


Saved to clut/c_chunk_000.parq: N_obj=986; took 2.32 sec


100%|██████████| 100000/100000 [03:59<00:00, 416.92it/s]


Saved to clut/a_chunk_000.parq: N_obj=100000; took 245.62 sec


100%|██████████| 100000/100000 [03:59<00:00, 416.67it/s]


...

Saved to clut/a_chunk_011.parq: N_obj=100000; took 240.92 sec


100%|██████████| 100000/100000 [03:54<00:00, 426.63it/s]


Saved to clut/a_chunk_012.parq: N_obj=100000; took 239.57 sec


100%|██████████| 39872/39872 [01:33<00:00, 426.55it/s]


Saved to clut/a_chunk_013.parq: N_obj=39872; took 95.74 sec


In [4]:
NO_SPK_FILE.sort()
ERROR_FILE.sort()
print("SPK file not found:", NO_SPK_FILE)
print("Calculation error :", ERROR_FILE)

SPK file not found: [20101955]
Calculation error : [1000084, 1000096, 1000143, 1000145, 1000237, 1000242, 1000365, 1000374, 1000468, 1000522, 20139754, 20196150, 20500577, 20546918, 20571179, 20582878, 20628041, 20628042, 54126829, 54405815, 54410854, 54414532]


Note that the calculation error is not a serious one, especially if the SPKID is smaller than 20,000,000 (any asteroid with good orbital constraints is given an ID larger than 20M. Comets, which may disappear/disrupted, or asteroids with uncertain orbits, will be given smaller numbers.)

For (101955) Bennu, there is a known problem for its BSP. Since it is a very faint target, I want to completely ignore it even though it is an interesting target (OSIRIS-REx mission target).